# 损失函数  


在统计学中损失函数是一种衡量损失和错误程度的函数。用来评价模型预测结果和样本真实结果的不一致程度，是一个非负实数函数，损失函数的值越小，就表明模型性能越好。  

机器学习中每个算法都有一个目标函数，算法的训练过程就是对目标函数优化的过程，在分类和回归等任务中，通常使用损失函数作为目标函数进行优化。

下面介绍常见的损失函数：

## 分类损失函数

### 1 softmax_cross_entropy_with_logits  


先看一下该函数的定义及说明

```
def softmax_cross_entropy_with_logits(
    _sentinel=None,  # pylint: disable=invalid-name
    labels=None,
    logits=None,
    dim=-1,
    name=None):
  
  """计算 `logits` 和 `labels` 之间的 softmax cross entropy.

  计算类别互斥的分类任务中的概率误差(每个样本只有一个类别)
  
  通常该函数输入的 logits 和 labels 的 shape 都是 `[batch_size, num_classes]`，
  但更高维度也是支持的，函数中的 `dim` 参数表示的就是分类维度

  **WARNING:** 该函数要求输入没有经过归一化的 logits，即没有经过 softmax
  处理的 logits, 为了提高计算的效率，函数内部将输入的 logits 做了一遍 softmax.
  所以不能输入 softmax 处理过的 logits.

  """
```

划重点：

- 1、函数输入维度为 [batch_size, num_classes] 的 logits 和 labels

- 2、输入的 logits 不能经过 softmax 处理， 函数内部做了softmax

- 3、函数输出是互斥分类任务的概率误差，不能是 muliti-label 分类任务  

- 4、在 inference 时候，不需要再添加 softmax 层，直接求 logits 的最大值就可以了，因为 softmax 不会影响到输出值的大小顺序。


#### cross entropy

cross entropy 可以度量两个概率分布 $y$ 和 $y^,$ 之间的距离，cross entropy 值越小，二者距离之间越接近。

$H = - \sum_i y_i \log(y_i^,)$  

$y^,$ 是模型预测的结果，$y$ 是数据标注。


#### softmax  

cross entropy 刻画的是两个概率分布之间的距离，但神经网络的输出却不是一个概率分布。为此通常需要使用 softmax 将神经网络的结果归一化到(0, 1) 区间内，得到分类的概率概率分布，就可以使用 cross entropy 计算分类概率与分类标注之间的距离。

$S_i = \frac {e^{v_i}}{\sum_i^C e^{v_i}}$  

softmax 输出表征了不同类别之间的相对概率。  

实际应用中，使用 softmax 需要注意数值溢出的问题。因为有指数运算，如果 $v_i$ 数值很大，经过指数运算后的数值往往可能有溢出的可能。所以，需要进行一些数值处理：即 $v$ 中的每个元素减去 $v$ 中的最大值。

```
scores = v
scores -= np.max(scores)
p = np.exp(scores) / np.sum(np.exp(scores))

```


### 2 sparse_softmax_cross_entropy_with_logits  


```
def sparse_softmax_cross_entropy_with_logits(
    _sentinel=None,  # pylint: disable=invalid-name
    labels=None,
    logits=None,
    name=None):
  
  """计算 `logits` 和 `labels` 之间的 sparse softmax cross entropy.

  **NOTE:** 该函数与 softmax_cross_entropy_with_logits 的区别在于
  传入的 label 是一个一维的 vector，每一个值就代表了 batch 中对应
  样本的类别，不需要转换为 one-hot 的格式。

  """

```

### 3 sigmoid_cross_entropy_with_logits

```
def sigmoid_cross_entropy_with_logits(  # pylint: disable=invalid-name
    _sentinel=None,
    labels=None,
    logits=None,
    name=None):
  
  """计算给定 `logits` 的 sigmoid_cross_entropy_with_logits.
  `logits` 和 `labels` 要具有相同的数据类型和维度.

  计算离散分类任务的概率误差，任务中每个类别之间是相互独立的，而不是互斥的，
  即该函数可用于计算 multilabel 任务的概率损失。
  
  对输入的 logits 先通过 sigmoid 函数计算，再计算与 labels 的交叉熵，
  并对交叉熵的计算方式进行了优化，使得的结果不至于溢出。
  
  对输入的 logits 通过 sigmoid 函数激活可将每个值都转换为 0-1 之间的值，
  可保证每个类别之间的独立性，所以可用于计算 multilabel 任务的损失。
  

  函数计算公式如下所示：

  z * -log(sigmoid(x)) + (1 - z) * -log(1 - sigmoid(x))
  
  注意这里计算了每个类别的 cross_entropy,将每个类别都作为一个二分类，
  而不是只计算正样本的 cross_entropy

  """
```


### 4 weighted_cross_entropy_with_logits  


```
def weighted_cross_entropy_with_logits(targets, logits, pos_weight, name=None):
  
  """ 计算加权 cross entropy.

  该函数类似 `sigmoid_cross_entropy_with_logits`，只是多了一个 `pos_weight`,
  这样可以通过 `pos_weight` 的值，来控制损失函数中正样本损失和负样本损失的比例，
  可以通过调高或调低 `pos_weight`在 precision 和 recall 之间做一个 trade off.

  常见的 cross-entropy 损失定义如下:

      targets * -log(sigmoid(logits)) +
          (1 - targets) * -log(1 - sigmoid(logits))

 `pos_weights > 1` 可以减少 false negative 的数量, 因此增加 recall.
  相反 `pos_weights < 1` 可以减少 false positive 的数量，因此增加 precision.
  
  weighted cross-entropy 损失定义如下:
      targets * -log(sigmoid(logits)) * pos_weight +
          (1 - targets) * -log(1 - sigmoid(logits))

  """
```


### 5 sampled_softmax_loss  


```
def sampled_softmax_loss(weights,
                         biases,
                         labels,
                         inputs,
                         num_sampled,
                         num_classes,
                         num_true=1,
                         sampled_values=None,
                         remove_accidental_hits=True,
                         partition_strategy="mod",
                         name="sampled_softmax_loss",
                         seed=None):
  
  """ 计算 sampled softmax training loss, 是 softmax cross entropy 计算的一种加速.

  训练数据的类别很多时，计算准确的概率分布会耗费很大的计算资源，该函数可以快速的训练一个具有大量类别的 softmax 分类器.
  而且该损失函数只在训练时候使用。
  
  计算时候先进行 class 采样，然后计算采样后的 logits 和 labels之间的 soft cross entropy 损失，
  在word2vec计算中，一般适用于 cbow 模型，通过上下文预测当前单词，所以输出只有一个类别正确.

  训练时设置参数 `partition_strategy="div"， 在测试时使用全量的 softmax loss.

  following example:

  if mode == "train":
    loss = tf.nn.sampled_softmax_loss(
        weights=weights,
        biases=biases,
        labels=labels,
        inputs=inputs,
        ...,
        partition_strategy="div")
  elif mode == "eval":
    logits = tf.matmul(inputs, tf.transpose(weights))
    logits = tf.nn.bias_add(logits, biases)
    labels_one_hot = tf.one_hot(labels, n_classes)
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(
        labels=labels_one_hot,
        logits=logits)

  """

```


### 7 nce_loss  

```
  def nce_loss(weights,
             biases,
             labels,
             inputs,
             num_sampled,
             num_classes,
             num_true=1,
             sampled_values=None,
             remove_accidental_hits=False,
             partition_strategy="mod",
             name="nce_loss"):
  
  """计算 noise-contrastive estimation 训练损失，是 sigmoid cross entropy 计算的一种加速.

  计算时候先进行 class 采样，然后计算采样后的 logits 和 labels之间的 sigmoid cross entropy 损失，
  在word2vec计算中，一般适用于 skip-gram 模型，通过当前单词预测上下文，所以输出有多个类别都是正确的.

  训练时设置参数 `partition_strategy="div"， 在测试时使用全量的 sigmoid loss.
  
  following example:

  if mode == "train":
    loss = tf.nn.nce_loss(
        weights=weights,
        biases=biases,
        labels=labels,
        inputs=inputs,
        ...,
        partition_strategy="div")
  elif mode == "eval":
    logits = tf.matmul(inputs, tf.transpose(weights))
    logits = tf.nn.bias_add(logits, biases)
    labels_one_hot = tf.one_hot(labels, n_classes)
    loss = tf.nn.sigmoid_cross_entropy_with_logits(
        labels=labels_one_hot,
        logits=logits)
    loss = tf.reduce_sum(loss, axis=1)

  Note: By default this uses a log-uniform (Zipfian) distribution for sampling,
  so your labels must be sorted in order of decreasing frequency to achieve
  good results.  For more details, see
  @{tf.nn.log_uniform_candidate_sampler}.

  Note: In the case where `num_true` > 1, we assign to each target class
  the target probability 1 / `num_true` so that the target probabilities
  sum to 1 per-example.

  """
```

sampled_softmax_loss 和 nce_loss 采用候选采样函数，按照一定原则，随机采样出类别子集，每次只评估所有类别的一个很小的子集。  

采样函数有：  

1、tf.nn.uniform_candidate_sampler  

均匀地采样出类别子集  
  
2、tf.nn.log_uniform_candidate_sampler  

按照 log-uniform (Zipfian) 分布采样。  

主要用于处理词作类别的情况。在语言学中，词按照出现频率从大到小排序之后，服从 Zipfian 分布。在使用这个函数之前，需要对类别按照出现频率从大到小排序。  

3、tf.nn.learned_unigram_candidate_sampler  

按照训练数据中类别出现分布进行采样  

4、tf.nn.fixed_unigram_candidate_sampler  

按照用户提供的概率分布进行采样  

如果类别服从均匀分布，就用 uniform_candidate_sampler; 如果词作类别，词服从 Zipfian, 就用 log_uniform_candidate_sampler; 如果能够通过统计或者其他渠道知道类别满足某些分布，就用 nn.fixed_unigram_candidate_sampler; 如果实在不知道类别分布，可以用 tf.nn.learned_unigram_candidate_sampler。  

## 回归损失函数  

### 1 MSE

## 序列损失函数